In [1]:
import numpy as np

In [2]:
K = 19
M = 0.5
rng = np.random.RandomState(seed=34)

In [3]:
def generate_sticks(M, rng, size):
    
    N = size[0]
    betas = rng.beta(1., M, size=size)
    
    sticks = np.hstack(
        [
            np.ones(shape=[N, 1]),
            np.cumprod(1 - betas[:, :-1], axis=1),
        ]
    )

    product = betas * sticks
    
    last_column = 1 - product.sum(axis=1)[..., np.newaxis] # adding last column to sampler seems to cause problems?
    
    return product

def stick_glueing(sticks):
    
    N = sticks.shape[0]
    
    # sticks = [w1, w2, ..., wK]
    # denominator = [1, -w1, -w2, ..., wK]
    denominator = np.hstack(
        [
            np.ones(shape=[N, 1]),
            - sticks,
        ]
    )
    
    denominator = np.cumsum(denominator, axis=1)
    
    # after cumsum, denominator = [1, 1 - w1, 1 - w1 - w2, ..., 1 - w1 - w2 - ... - wK]
    
    # output is now [w1/1, w2/(1 - w1), w3/(1 - w1 - w2), ..., wK/(1 - w1 - ... - wK-1)]
    
    return sticks/(denominator[:, :-1])

In [4]:
sticks = generate_sticks(M, rng, size=[1000, K])

In [5]:
denominator = np.hstack(
    [
            np.ones(shape=[1000, 1]),
            - sticks,
    ]
)
    
denominator = np.cumsum(denominator, axis=1)

In [6]:
denominator.dtype

dtype('float64')

#### $\sum_{\ell < h} w_\ell$

In [7]:
np.cumsum(sticks[103])

array([0.81991071, 0.98505847, 0.99698764, 0.99826655, 0.99860702,
       0.99871993, 0.99999997, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

#### $1 - \sum_{\ell < h} w_\ell$

In [8]:
1 - np.cumsum(sticks[103])

array([1.80089293e-01, 1.49415257e-02, 3.01235691e-03, 1.73345246e-03,
       1.39298192e-03, 1.28006867e-03, 2.95100887e-08, 4.93831642e-11,
       2.51096921e-11, 3.17523785e-13, 1.11022302e-16, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

#### $1 - \sum_{\ell < h} w_\ell$, but with a column of ones in front

In [9]:
denominator[103, 1:] # yields negative beta values, hence -inf log-likelihood

array([ 1.80089293e-01,  1.49415257e-02,  3.01235691e-03,  1.73345246e-03,
        1.39298192e-03,  1.28006867e-03,  2.95100888e-08,  4.93832688e-11,
        2.51097428e-11,  3.17532423e-13,  1.52659119e-16, -1.35175325e-17,
       -1.41372713e-17, -1.41754840e-17, -1.41839428e-17, -1.41894030e-17,
       -1.41895716e-17, -1.41896098e-17, -1.41896114e-17])

In [10]:
1 - denominator[103, 1:]

array([0.81991071, 0.98505847, 0.99698764, 0.99826655, 0.99860702,
       0.99871993, 0.99999997, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

The same numerical imprecision can be observed for recovering beta realizations (pretty much same code as above).


Some variable names are used twice...

In [ ]:
betas = rng.beta(1., M, size=[3, 19])

sticks = np.hstack(
        [
            np.ones(shape=[3, 1]),
            np.cumprod(1 - betas[:, :-1], axis=1),
        ]
    )
sticks = sticks*betas

denominator = np.hstack(
        [
            np.ones(shape=[3, 1]),
            - sticks,
        ]
    )
denominator = np.cumsum(denominator, axis=1)

recovered_betas = sticks/denominator[:, :-1]

In [ ]:
betas[2]

In [ ]:
recovered_betas[2] # precision lost starting at [13:]

### An attempt to find a reasonable upper bound for `K`

For numerical stability purposes. Recall the definition of our stick-breaking weights $w_h$ with respect to Beta random variables which we denote by $v_h \stackrel{\text{i.i.d.}}{\sim} \text{Beta}(1, M)$:

\begin{align*}
    w_h = v_h \prod_{\ell < h} (1 - v_\ell)
\end{align*}

According to my naked eye, numerical instability issues seem to occur when weights fall below a value of $10^{-4}$ (see above). To be safe, I require that all $\mathbb{E}[w_h] > 10^{-3}$ for all $h = 1, \dots, K$, especially for $h = K$ since the weights form a strictly decreasing sequences in their expectation. A more formal argument can perhaps be divised using CLT and the Delta Method.

\begin{align*}
    \mathbb{E}(w_K) &= \mathbb{E}(v_K) \prod_{\ell < K} \left(1 - \mathbb{E}(v_\ell)\right) \qquad \text{by independence}\\
    &= \frac{1}{1 + M} \left(\frac{M}{M + 1}\right)^{K - 1}\\
    &\stackrel{\text{set}}{>} 10^{-3}\\
    \Leftrightarrow K &< \frac{\log(10^{-3}) + \log(M+1) + \log\left(\frac{M}{M+1}\right)}{\log\left(\frac{M}{M+1}\right)}\\
    &= \frac{\log(10^{-3}) + \log(M+1)}{\log\left(\frac{M}{M+1}\right)} + 1
\end{align*}

A quick attempt at implementing this shows that it still doesn't work very well for $M = 0.2, 0.01$...